# Bilateral International Migration prediction using Deep Learning #


Ludwig-Maximilians-Universität M.Economics Machine Learning Applications - WS2020/21 Term Paper

Code by: Yunting LIU,  15.03.2021

## Note ##

For an introduction to the research topic and methodology see accompanying document "Notes to exam project_bilateral migration prediction". This notebook is an extension to the base notebook "Exam_bilateral_migration_predictions_deep_learning"


# Initialising

In [1]:
import os
import wget
from pathlib import Path

import pandas as pd
import numpy as np

from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.metrics import Metric

import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, mean_absolute_error, mean_squared_error, r2_score 
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as kb
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# Set a random seed to ensure result reproducibility
import random as python_random
random_seed = 123
np.random.seed(random_seed)
python_random.seed(random_seed)
tf.random.set_seed(random_seed)



In [3]:
# Change current working directory for convenience

os.chdir('/Users/jacqueline/Documents/Study/LMU/2020WS/Machine Learning Applications/exam_paper')
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /Users/jacqueline/Documents/Study/LMU/2020WS/Machine Learning Applications/exam_paper


# Data

Prepared in main notebook: Exam_bilateral_migration_predictions_deep_learning_v4.ipynb

In [4]:
data_modelling = pd.read_csv('./data/data_modelling.csv', delimiter=";", index_col=False)

target = 'fwd_log_mig' 

categorical_columns = ['pair_id','iso3_o','iso3_d']
categorical_dims = {'pair_id': 2627, 'iso3_o': 101, 'iso3_d': 35}

features = ['pair_id','year','iso3_o','iso3_d','o_sl_uem_totl_zs','o_sp_pop_0014_to_zs','sfi_sfi',
            'pol4_autoc','o_IT_CEL_SETS_P2','o_IT_NET_USER_P2','o_number_weather','o_number_nonweather',
            'mig_stock_tot','o_French','o_Spanish','o_English','d_GDP','d_pop','o_GDP','o_pop']


# Split into train, validation and test set

X_train = data_modelling.loc[data_modelling["Set"]==1][features].values
y_train = np.array(data_modelling.loc[data_modelling["Set"]==1][target], dtype = 'float32').reshape(-1, 1)

X_valid = data_modelling.loc[data_modelling["Set"]==2][features].values
y_valid = np.array(data_modelling.loc[data_modelling["Set"]==2][target], dtype = 'float32').reshape(-1, 1)

X_test = data_modelling.loc[data_modelling["Set"]==0][features].values
y_test = np.array(data_modelling.loc[data_modelling["Set"]==0][target], dtype = 'float32').reshape(-1, 1)


## Defining CPC and R2 as custom metric, defining CPC as loss function 

In [ ]:
# Define Common Part of Commuters (CPC) and R_squared as custom metrics

class CPC(Metric):
    def __init__(self):
        self._name = "cpc"
        self._maximize = True

    def __call__(self, y_true, y_score):
        cpc = 2 * np.sum(np.minimum(y_true, y_score)) / (np.sum(y_true) + np.sum(y_score))
        return cpc

# Coefficient of determination r^2

class r_squared(Metric):
    def __init__(self):
        self._name = "r_squared"
        self._maximize = True

    def __call__(self, y_true, y_score):
        SS_res = np.sum(np.square(y_true - y_score))
        SS_tot = np.sum(np.square(y_true - np.mean(y_true)))
        r_squared = (1 - SS_res / (SS_tot))          
        return r_squared

custom_metrics = ['mae', 'mse', 'rmse', r_squared, CPC]

In [ ]:
# Again for Keras models
# Common Part of Commuters
def cpc_keras(y_true, y_pred):
    return (2.0 * kb.sum(kb.minimum(y_true, y_pred))) / (kb.sum(y_true) + kb.sum(y_pred))

# Coefficient of determination r^2
def r_squared_keras(y_true, y_pred):
    SS_res = kb.sum(kb.square(y_true - y_pred))
    SS_tot = kb.sum(kb.square(y_true - kb.mean(y_true)))
    r_squared = (1 - SS_res / (SS_tot + kb.epsilon()))
    return r_squared

rmse_keras = keras.metrics.RootMeanSquaredError()

custom_metrics_keras = ['mae', 'mse', rmse_keras, r_squared_keras, cpc_keras] # to be passed to model.compile(), metrics parameter


In [ ]:
def cpc_loss(y_pred, y_true):
    cpc = 2 * torch.sum(torch.minimum(y_true, y_pred)) / (torch.sum(y_true) + torch.sum(y_pred))
    return 1-cpc

In [ ]:
def cpc_loss_xgb(y_pred, y_true):
    print(y_pred)
    print(y_pred.shape)
    print(y_true)
    print(y_true.shape)
    cpc = 2 * sum(np.minimum(y_true, y_pred)) / (sum(y_true) + sum(y_pred))    
    return cpc

In [ ]:
def cpc_loss_keras(y_true, y_pred):
#    return 1.0 - cpc_keras(y_true, y_pred)
    return 1.0 - (2.0 * kb.sum(kb.minimum(y_true, y_pred))) / (kb.sum(y_true) + kb.sum(y_pred))

# Hyper-parameter tuning: TabNet

In [ ]:
#1. TabNet

# define hyper-parameter grid search space

# For Tabnet, approximate Range of Model Hyperparameters are given as:

# batch size: 256 to 32768
# virtual batch size: 128 to 2048

p_tabnet = {'batch_size': [256, 512, 1024, 2048, 4096, 8192, 16384, 32768],
            'virtual_batch_size':[128, 256, 512, 1024, 2048],
            'layer_width':[4,8,12,16]
           }


In [ ]:
gridsearch_tabnet_header = np.array(["batch_size", "virtual_batch_size", "best_epoch", "best_cost"])

gridsearch_tabnet = np.empty([0,5])

# Define categorical features for categorical embeddings
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

for batch_size in p_tabnet.get('batch_size'):
    for virtual_batch_size in p_tabnet.get('virtual_batch_size'):
        if virtual_batch_size <= batch_size:            
                for layer_width in p_tabnet.get('layer_width'):
                    print("fitting batch_size ",batch_size," and virtual_batch_size ",virtual_batch_size," with layer_width ",layer_width)

                    modelTabnet = TabNetRegressor(n_d=layer_width, n_a=layer_width, n_steps=5, cat_dims=cat_dims, cat_idxs=cat_idxs, seed=random_seed, optimizer_params=dict(lr=2e-3), clip_value=1., device_name='cpu')
                    modelTabnet.fit(
                        X_train=X_train, y_train=y_train,
                        eval_set=[(X_train, y_train), (X_valid, y_valid)],
                        eval_name = ['train', 'valid'],
                        eval_metric= ['mae', 'mse', 'rmse', r_squared, CPC],
                        loss_fn = cpc_loss,
                        max_epochs=2000,
                        patience=50,
                        batch_size=batch_size, 
                        virtual_batch_size=virtual_batch_size,
                        num_workers=0,
                        drop_last=False)
                    result_experiment = [batch_size, virtual_batch_size, layer_width, modelTabnet.best_epoch, modelTabnet.best_cost]
                    #print(result_experiment)
                    gridsearch_tabnet = np.vstack((gridsearch_tabnet, np.array(result_experiment)))

            

fitting batch_size  256  and virtual_batch_size  128  with layer_width  4
Device used : cpu
epoch 0  | loss: 0.48561 | train_mae: 357.76104736328125| train_mse: 248264.6875| train_rmse: 498.2616271972656| train_r_squared: -37175.27344| train_cpc: 0.01351 | valid_mae: 364.0142517089844| valid_mse: 256839.4375| valid_rmse: 506.79327392578125| valid_r_squared: -38129.72656| valid_cpc: 0.01401 |  0:00:04s
epoch 1  | loss: 0.34443 | train_mae: 92.20538330078125| train_mse: 20931.39453125| train_rmse: 144.67686462402344| train_r_squared: -3133.36133| train_cpc: 0.05305 | valid_mae: 103.56326293945312| valid_mse: 26167.19921875| valid_rmse: 161.76278686523438| valid_r_squared: -3883.81763| valid_cpc: 0.0502  |  0:00:09s
epoch 2  | loss: 0.31646 | train_mae: 61.17586135864258| train_mse: 16792.912109375| train_rmse: 129.58746337890625| train_r_squared: -2513.64648| train_cpc: 0.07748 | valid_mae: 62.5476188659668| valid_mse: 18189.41796875| valid_rmse: 134.86814880371094| valid_r_squared: -269

epoch 26 | loss: 0.21171 | train_mae: 1.8880499601364136| train_mse: 5.3844499588012695| train_rmse: 2.3204400539398193| train_r_squared: 0.19371 | train_cpc: 0.75078 | valid_mae: 1.967919945716858| valid_mse: 5.87129020690918| valid_rmse: 2.4230799674987793| valid_r_squared: 0.12834 | valid_cpc: 0.74908 |  0:02:03s
epoch 27 | loss: 0.21053 | train_mae: 1.9657100439071655| train_mse: 5.808810234069824| train_rmse: 2.4101500511169434| train_r_squared: 0.13016 | train_cpc: 0.74347 | valid_mae: 2.017159938812256| valid_mse: 6.168600082397461| valid_rmse: 2.4836699962615967| valid_r_squared: 0.0842  | valid_cpc: 0.74469 |  0:02:07s
epoch 28 | loss: 0.20983 | train_mae: 2.0661699771881104| train_mse: 6.37637996673584| train_rmse: 2.5251500606536865| train_r_squared: 0.04517 | train_cpc: 0.73511 | valid_mae: 2.0773301124572754| valid_mse: 6.415460109710693| valid_rmse: 2.5328800678253174| valid_r_squared: 0.04755 | valid_cpc: 0.73966 |  0:02:12s
epoch 29 | loss: 0.2072  | train_mae: 2.134210

epoch 52 | loss: 0.17749 | train_mae: 2.5988399982452393| train_mse: 9.47974967956543| train_rmse: 3.0789198875427246| train_r_squared: -0.41954| train_cpc: 0.69288 | valid_mae: 2.606750011444092| valid_mse: 9.568559646606445| valid_rmse: 3.0933101177215576| valid_r_squared: -0.42056| valid_cpc: 0.6984  |  0:03:59s
epoch 53 | loss: 0.17768 | train_mae: 2.588939905166626| train_mse: 9.439499855041504| train_rmse: 3.0723800659179688| train_r_squared: -0.41351| train_cpc: 0.69347 | valid_mae: 2.5894899368286133| valid_mse: 9.500539779663086| valid_rmse: 3.0822999477386475| valid_r_squared: -0.41046| valid_cpc: 0.69975 |  0:04:03s
epoch 54 | loss: 0.17686 | train_mae: 2.8964900970458984| train_mse: 11.635029792785645| train_rmse: 3.411020040512085| train_r_squared: -0.74228| train_cpc: 0.67287 | valid_mae: 2.9037699699401855| valid_mse: 11.722709655761719| valid_rmse: 3.423840045928955| valid_r_squared: -0.74037| valid_cpc: 0.67878 |  0:04:08s
epoch 55 | loss: 0.17622 | train_mae: 2.943850

epoch 7  | loss: 0.25046 | train_mae: 1.6595900058746338| train_mse: 4.289030075073242| train_rmse: 2.071000099182129| train_r_squared: 0.35774 | train_cpc: 0.75711 | valid_mae: 1.7560900449752808| valid_mse: 4.807559967041016| valid_rmse: 2.192610025405884| valid_r_squared: 0.28626 | valid_cpc: 0.75702 |  0:00:39s
epoch 8  | loss: 0.24349 | train_mae: 1.6559799909591675| train_mse: 4.352369785308838| train_rmse: 2.0862300395965576| train_r_squared: 0.34826 | train_cpc: 0.75997 | valid_mae: 1.747290015220642| valid_mse: 4.745629787445068| valid_rmse: 2.178450107574463| valid_r_squared: 0.29546 | valid_cpc: 0.75809 |  0:00:44s
epoch 9  | loss: 0.23681 | train_mae: 1.60316002368927| train_mse: 4.484580039978027| train_rmse: 2.117680072784424| train_r_squared: 0.32846 | train_cpc: 0.76163 | valid_mae: 1.6445900201797485| valid_mse: 4.305469989776611| valid_rmse: 2.0749599933624268| valid_r_squared: 0.3608  | valid_cpc: 0.76306 |  0:00:49s
epoch 10 | loss: 0.23638 | train_mae: 1.6179300546

epoch 33 | loss: 0.18206 | train_mae: 1.9696999788284302| train_mse: 5.517199993133545| train_rmse: 2.348870038986206| train_r_squared: 0.17383 | train_cpc: 0.74516 | valid_mae: 1.9553200006484985| valid_mse: 5.508580207824707| valid_rmse: 2.3470399379730225| valid_r_squared: 0.18219 | valid_cpc: 0.7504  |  0:02:48s
epoch 34 | loss: 0.18102 | train_mae: 1.8548500537872314| train_mse: 4.896450042724609| train_rmse: 2.212790012359619| train_r_squared: 0.26678 | train_cpc: 0.75486 | valid_mae: 1.8416399955749512| valid_mse: 4.899350166320801| valid_rmse: 2.2134499549865723| valid_r_squared: 0.27264 | valid_cpc: 0.75958 |  0:02:53s
epoch 35 | loss: 0.17971 | train_mae: 1.7430700063705444| train_mse: 4.588510036468506| train_rmse: 2.142080068588257| train_r_squared: 0.3129  | train_cpc: 0.76579 | valid_mae: 1.7890199422836304| valid_mse: 4.769790172576904| valid_rmse: 2.183990001678467| valid_r_squared: 0.29187 | valid_cpc: 0.76511 |  0:02:57s
epoch 36 | loss: 0.17717 | train_mae: 1.7615900

...

epoch 610| loss: 0.05912 | train_mae: 0.3408200144767761| train_mse: 0.26987001299858093| train_rmse: 0.5194900035858154| train_r_squared: 0.95959 | train_cpc: 0.94636 | valid_mae: 0.46362000703811646| valid_mse: 0.5786200165748596| valid_rmse: 0.760670006275177| valid_r_squared: 0.9141  | valid_cpc: 0.92932 |  3:12:03s
epoch 611| loss: 0.06033 | train_mae: 0.3372099995613098| train_mse: 0.27316001057624817| train_rmse: 0.5226500034332275| train_r_squared: 0.9591  | train_cpc: 0.9467  | valid_mae: 0.4621799886226654| valid_mse: 0.5752300024032593| valid_rmse: 0.7584400177001953| valid_r_squared: 0.9146  | valid_cpc: 0.92919 |  3:12:06s
epoch 612| loss: 0.06013 | train_mae: 0.3576900064945221| train_mse: 0.28641000390052795| train_rmse: 0.5351700186729431| train_r_squared: 0.95711 | train_cpc: 0.94367 | valid_mae: 0.4865399897098541| valid_mse: 0.6364499926567078| valid_rmse: 0.7977799773216248| valid_r_squared: 0.90551 | valid_cpc: 0.9259  |  3:12:10s
epoch 613| loss: 0.06003 | train_m

epoch 636| loss: 0.05846 | train_mae: 0.3383300006389618| train_mse: 0.2601099908351898| train_rmse: 0.5100100040435791| train_r_squared: 0.96105 | train_cpc: 0.94685 | valid_mae: 0.45763999223709106| valid_mse: 0.5519999861717224| valid_rmse: 0.7429699897766113| valid_r_squared: 0.91805 | valid_cpc: 0.93021 |  3:13:49s
epoch 637| loss: 0.05891 | train_mae: 0.34703001379966736| train_mse: 0.27448999881744385| train_rmse: 0.5239099860191345| train_r_squared: 0.9589  | train_cpc: 0.94542 | valid_mae: 0.4631600081920624| valid_mse: 0.580590009689331| valid_rmse: 0.7619699835777283| valid_r_squared: 0.9138  | valid_cpc: 0.92919 |  3:13:54s
epoch 638| loss: 0.05806 | train_mae: 0.32649001479148865| train_mse: 0.25380998849868774| train_rmse: 0.5037999749183655| train_r_squared: 0.96199 | train_cpc: 0.94803 | valid_mae: 0.449290007352829| valid_mse: 0.5852500200271606| valid_rmse: 0.7650200128555298| valid_r_squared: 0.91311 | valid_cpc: 0.93048 |  3:13:58s
epoch 639| loss: 0.05892 | train_m

epoch 662| loss: 0.0607  | train_mae: 0.32179999351501465| train_mse: 0.25224998593330383| train_rmse: 0.502240002155304| train_r_squared: 0.96223 | train_cpc: 0.94872 | valid_mae: 0.44861000776290894| valid_mse: 0.5636100172996521| valid_rmse: 0.7507399916648865| valid_r_squared: 0.91633 | valid_cpc: 0.93062 |  3:15:39s
epoch 663| loss: 0.05848 | train_mae: 0.3442699909210205| train_mse: 0.2690100073814392| train_rmse: 0.5186600089073181| train_r_squared: 0.95972 | train_cpc: 0.94593 | valid_mae: 0.46219998598098755| valid_mse: 0.5533199906349182| valid_rmse: 0.7438499927520752| valid_r_squared: 0.91785 | valid_cpc: 0.92959 |  3:15:44s
epoch 664| loss: 0.05867 | train_mae: 0.3323400020599365| train_mse: 0.2589699923992157| train_rmse: 0.5088899731636047| train_r_squared: 0.96122 | train_cpc: 0.94765 | valid_mae: 0.48403000831604004| valid_mse: 0.7869300246238708| valid_rmse: 0.8870900273323059| valid_r_squared: 0.88317 | valid_cpc: 0.92642 |  3:15:48s
epoch 665| loss: 0.05829 | train_

epoch 688| loss: 0.05835 | train_mae: 0.34185001254081726| train_mse: 0.2742300033569336| train_rmse: 0.5236700177192688| train_r_squared: 0.95894 | train_cpc: 0.94626 | valid_mae: 0.4748600125312805| valid_mse: 0.6342999935150146| valid_rmse: 0.7964299917221069| valid_r_squared: 0.90583 | valid_cpc: 0.92763 |  3:17:49s
epoch 689| loss: 0.0582  | train_mae: 0.32642000913619995| train_mse: 0.25102001428604126| train_rmse: 0.5010200142860413| train_r_squared: 0.96241 | train_cpc: 0.94835 | valid_mae: 0.4619700014591217| valid_mse: 0.6162099838256836| valid_rmse: 0.7849900126457214| valid_r_squared: 0.90852 | valid_cpc: 0.92921 |  3:52:55s
epoch 690| loss: 0.05855 | train_mae: 0.3364900052547455| train_mse: 0.2653999924659729| train_rmse: 0.5151699781417847| train_r_squared: 0.96026 | train_cpc: 0.94713 | valid_mae: 0.4687899947166443| valid_mse: 0.6037099957466125| valid_rmse: 0.7769899964332581| valid_r_squared: 0.91037 | valid_cpc: 0.9286  |  3:52:58s
epoch 691| loss: 0.05993 | train_m

epoch 1  | loss: 0.58493 | train_mae: 80.87787628173828| train_mse: 17433.638671875| train_rmse: 132.0364990234375| train_r_squared: -2609.59155| train_cpc: 12.91136| valid_mae: 87.36873626708984| valid_mse: 20245.62890625| valid_rmse: 142.28713989257812| valid_r_squared: -3004.69312| valid_cpc: -28.27734|  0:00:07s
epoch 2  | loss: 0.37282 | train_mae: 31.099700927734375| train_mse: 7444.49853515625| train_rmse: 86.28150939941406| train_r_squared: -1113.77283| train_cpc: -0.14464| valid_mae: 35.69512939453125| valid_mse: 9268.6484375| valid_rmse: 96.27381896972656| valid_r_squared: -1375.03613| valid_cpc: -0.10136|  0:00:10s
epoch 3  | loss: 0.32238 | train_mae: 40.30514144897461| train_mse: 4822.54052734375| train_rmse: 69.44451141357422| train_r_squared: -721.14893| train_cpc: -0.26485| valid_mae: 29.549400329589844| valid_mse: 2879.933349609375| valid_rmse: 53.665008544921875| valid_r_squared: -426.55881| valid_cpc: -0.17408|  0:00:13s


In [ ]:
pd.DataFrame(gridsearch_tabnet)

# Hyper-parameter tuning: ANN

In [ ]:
#2. ANN

# define hyper-parameter grid search space

p_ann = {'batch_size': [128, 256, 512, 1024, 2048, 4096],
             'n_layer': [1,2,3],
             'size_hidden_layer': [5,10,15,20,30,40,50]
         
           }

In [ ]:
def defineANN(n_features = 20,l=cpc_loss_keras,size_hidden_layer=20,dropout=0.15,n_layer=2):  
    model = Sequential()
    if dropout > 0.0:
        model.add(Dropout(dropout,input_shape=(n_features,)))
    for i in range(n_layer):
        model.add(Dense(size_hidden_layer, activation='relu', input_shape=(n_features,)))
    model.add(Dense(1))
    opt = keras.optimizers.Adam(learning_rate=0.002, clipnorm = 2)
    model.compile(optimizer=opt, loss=l, metrics = custom_metrics_keras)
#  model.compile(optimizer='adam'(lr=0.001), loss=l, metrics = custom_metrics_keras)

    return model

In [ ]:
gridsearch_ann_header = np.array(["batch_size", "virtual_batch_size", "best_epoch", "best_loss"])

gridsearch_ann = np.empty([0,5])

for batch_size in p_ann.get('batch_size'):
    for n_layer in p_ann.get('n_layer'):
        for size_hidden_layer in p_ann.get('size_hidden_layer'):
         
            print("fitting batch_size",batch_size,", n_layer",n_layer,"and size_hidden_layer",size_hidden_layer)

            modelAnn = defineANN(size_hidden_layer=size_hidden_layer,n_layer=n_layer)
            historyAnn = modelAnn.fit(X_train, y_train, epochs=2000, batch_size=batch_size, verbose=1, validation_data=(X_valid, y_valid))
            
            historyAnn_cpc = historyAnn.history.get('val_cpc_keras')
            modelAnn_best_epoch = 2000
            modelAnn_best_cpc = historyAnn_cpc[-1]
            
            result_experiment = [batch_size, n_layer, size_hidden_layer, modelAnn_best_epoch, modelAnn_best_loss]
            print(result_experiment)
            gridsearch_ann = np.vstack((gridsearch_ann, np.array(result_experiment)))

In [ ]:
pd.DataFrame(gridsearch_ann)

## Training Models - Exploration with LSTM (Failed)

import tensorflow as tf

# convert data to 3-d array (sample_id, year, features) to work wth LSTM model
data_modelling_indexed = data_modelling.set_index(['year', 'pair_id'])

# Simple feature pre-processing
nunique = data_modelling_indexed.nunique()
types = data_modelling_indexed.dtypes

categorical_columns = []
categorical_dims =  {}

for col in data_modelling_indexed.columns[data_modelling_indexed.dtypes == object]:
    print(col, data_modelling_indexed[col].nunique())
    l_enc = LabelEncoder()
    data_modelling_indexed[col] = data_modelling_indexed[col].fillna("VV_likely")
    data_modelling_indexed[col] = l_enc.fit_transform(data_modelling_indexed[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)


features_3d = features.copy()
features_3d.remove('pair_id')
features_3d.remove('year')
print(features)
print(features_3d)

X_indexed = data_modelling_indexed[features_3d]
y_indexed = data_modelling_indexed[target]
print(X_indexed.shape)
print(y_indexed.shape)

X_3d = tf.convert_to_tensor(np.transpose(np.array(X_indexed.to_xarray().to_array()), (2,1,0)))
y_3d = tf.convert_to_tensor(np.transpose(np.array(y_indexed.to_xarray())[:,np.newaxis], (2,0,1)))


print(X_3d.shape)
print(y_3d.shape)

# Define the lstm model
#
# @data              : a dataframe like the transformData return
# @l                 : loss function to build model
# @size_hidden_layer : size of the LSTM's hidden layer
# @dropout           : value of the model's dropout
# @returns           : the defined model

# data is perpared as a 3D array (samples, time steps, features)
# n_timeperiod = 10 for years 2004 until 2013; 2014 is reserved for testing so not counted
def defineLSTM(n_features = 18, n_timeperiod = 11, l = cpc_loss_keras, size_hidden_layer = 50, dropout = 0):
  n_features = n_features
  model = Sequential()
  if dropout > 0.0:
    model.add(Dropout(dropout, input_shape=(1,n_features)))
  model.add(LSTM(size_hidden_layer, activation='relu', input_shape=(n_timeperiod, n_features)))
  model.add(Dense(1))
  opt = keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1)
  model.compile(optimizer=opt, loss=l, metrics = custom_metrics_keras)

  return model

modelLstm = defineLSTM()

history=modelLstm.fit(X_3d, y_3d, epochs=1000, batch_size=1280, verbose=1, validation_split=0.2)
#predictions = model.predict(X_test)
